https://076923.github.io/posts/Python-pytorch-13/  
https://sanghyu.tistory.com/24  
https://journal-home.s3.ap-northeast-2.amazonaws.com/site/2020kics/presentation/0565.pdf  

https://2bman.tistory.com/3

In [2]:
# Standard
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# from torch.utils.data import DataLoader
# from loader.custom_dataset import DS, collate_func

###### in_channels: input의 feature dimension
- out_channels: 내가 output으로 내고싶은 dimension
- kernel_size: time step을 얼마만큼 볼 것인가(=frame size = filter size)
- stride: kernel을 얼마만큼씩 이동하면서 적용할 것인가 (Default: 1)
- dilation: kernel 내부에서 얼마만큼 띄어서 kernel을 적용할 것인가 (Default: 1)
- padding: 한 쪽 방향으로 얼마만큼 padding할 것인가 (그 만큼 양방향으로 적용) (Default: 0)
- groups: kernel의 height를 조절 (Default: 1)
- bias: bias term을 둘 것인가 안둘 것인가 (Default: True)
- padding_mode: 'zeros', 'reflect', 'reflect', 'replicate', 'circular' (Default: 'zeros')  
  
- Input : [Batch_size, Feature_dimension, Time_step]
- Out : [Batch size, Feature dimension(Channel_dimension), kernel로 변경된 Time_step]

In [3]:
from glob import glob
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from loader.custom_dataset import FinDataset, FinCollate
from model.model import FinCNN

In [4]:
import torch
import torch.nn as nn
from torch.optim import Adam

BATCH_SIZE = 32
learning_rate = 0.001
num_epochs = 1
num_classes= 2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = FinCNN(num_classes=num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_set, valid_set = train_test_split(sorted(glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet"))),shuffle=True, test_size=0.2, random_state=42)

train_loader = enumerate(tqdm(DataLoader(FinDataset(data_paths=train_set), batch_size=BATCH_SIZE, collate_fn=FinCollate())))
valid_loader = enumerate(tqdm(DataLoader(FinDataset(data_paths=valid_set), batch_size=BATCH_SIZE, collate_fn=FinCollate())))

/home/ubuntu/Fin/Finance/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def metric(logit, target, batch_size):
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [6]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    model = model.train()

    ## training step
    for i, (x, y) in train_loader:
        
        x = x.to(device)
        y = y.to(device)

        ## forward + backprop + loss
        logits = model(x)
        loss = criterion(logits, y)
        
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += metric(logits, y, BATCH_SIZE)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i))      

: 

In [12]:
test_acc = 0.0
for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        
print('Test Accuracy: %.2f'%( test_acc/i))

tensor([[[    0.0000,     0.0000,     2.1594,  ...,     0.0000,
              0.0000,     0.0000],
         [    3.3632,     3.3632,     3.4084,  ...,     0.0000,
              0.0000,     0.0000],
         [    3.0000,     3.0000,     3.0000,  ...,     0.0000,
              0.0000,     0.0000]],

        [[    2.7579,     1.7587,     1.1750,  ...,     0.0000,
              0.0000,     0.0000],
         [    4.0224,     3.9841,     3.9617,  ...,     0.0000,
              0.0000,     0.0000],
         [    5.0000,     5.0000,     5.0000,  ...,     0.0000,
              0.0000,     0.0000]],

        [[    0.0000,     3.2224,     0.0000,  ...,     1.3226,
              1.3226,     1.7462],
         [    3.8259,     3.9390,     3.8259,  ...,     3.8720,
              3.8720,     3.8869],
         [    1.0000,     5.0000,     1.0000,  ...,     5.0000,
              1.0000,     5.0000]],

        [[    0.0000,     0.6309,     0.0000,  ...,     0.0000,
              0.0000,     0.0000],
    

In [ ]:
for x, y in test(sorted(glob(os.path.join(os.path.abspath(os.path.pardir), "data", "raw", "*.parquet")))[0]):
    print(y)
    break